In [ ]:
"""--------------------------------------------------------------------------
ICA
--------------------------------------------------------------------------"""
def kurt(Y):
    return torch.mean(Y**4, dim=0) - 3
def kurt_loss(Y):
    # Y の分散共分散が単位行列であることが前提=>白色化後なのでok
    return -torch.mean(torch.abs(kurt(Y)))

def ICA(X, loss_func = kurt_loss):
    X = Variable(torch.from_numpy(X), requires_grad =False).float()
    W = Variable((torch.randn((2, 2))), requires_grad =True)
    W.data = W.data / torch.norm(W.data)
    hist_loss = []
    
    for i in range(10):
        Y = torch.mm(X, W)
        loss = loss_func(Y)
        hist_loss.append(loss.data.numpy())
        loss.backward()

        W.data = W.data - W.grad.data * 0.1
        W.grad.data.zero_()
        W.data = W.data / torch.norm(W.data)
    print("y_type="+str(type(Y)))
    return Y, W, hist_loss

#Y, W, hist_loss = ICA(X, loss_func=kurt_loss)
#plt.plot(Y.data.numpy())
#plt.show()
#plt.plot(hist_loss)

def FDICA(white_X1,white_X2, loss_func = kurt_loss):
    
    gyou = int(white_X1.shape[0])
    retu = int(white_X1.shape[1])
    
    Y1 = np.zeros_like((white_X1), dtype='float')
    Y2 = np.zeros_like((white_X1), dtype='float')
    Wr = np.zeros([2,2,gyou],  dtype='float')
    
    Y1=torch.from_numpy(Y1)
    Y2=torch.from_numpy(Y2)
    Wr=torch.from_numpy(Wr)
    
    print("FDICA start")
    
    
    for f in range(gyou):
        X1_temp = white_X1[ f ,:]
        X2_temp = white_X1[ f ,:]
        #X = [X1_temp,X2_temp]
    
       # X1_temp = X1_temp.T
       # X2_temp = X2_temp.T
        X = np.array((X1_temp,X2_temp), dtype='float')
        #X = np.array((X1_temp,X2_temp))
        #X = np.array(X)
        X = X.T
        
        #print("X_fdica_type = "+str(type(X)))
        #print("X_fdica_size = "+str(X.shape))
        
        X = Variable(torch.from_numpy(X), requires_grad =False).float()
        W = Variable((torch.randn((2, 2))), requires_grad =True)
        W.data = W.data / torch.norm(W.data)
        hist_loss = []
    
        for i in range(10):
            Y = torch.mm(X, W)
            loss = loss_func(Y)
            hist_loss.append(loss.data.numpy())
            loss.backward()

            W.data = W.data - W.grad.data * 0.1
            W.grad.data.zero_()
            W.data = W.data / torch.norm(W.data)
        
        Y.detach().numpy()
        #print("Y_fdica_type = "+str(type(Y)))
        Y1[f, :] = Y[:,0]
        Y2[f, :] = Y[:,1]
        Wr[:,:,f] = W[:,:]
    
    Y1 = Y1.detach().numpy()
    Y2 = Y2.detach().numpy()
    print("FDICA finished")
    return Y1,Y2, Wr, hist_loss